In [1]:
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 15
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

from matplotlib.colors import LinearSegmentedColormap


import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
    
    
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

saveFigures is set to: True
Done loading packages


In [2]:

latestsubdir = list(os.walk(path_dash))[0][1][-1]
latestdir = path_dash + latestsubdir
latestdir

# dfAge = pd.read_csv(latestdir+'\\Regionalt_DB\\18_fnkt_alder_uge_testede_positive_nyindlagte.csv',delimiter=';',encoding='latin1')

# dfCurAdmitted = pd.read_csv(latestdir+'\\Gennembruds_DB\\01_indlagte_pr_vaccinationsstatus.csv',delimiter=';',encoding='latin1')
df = pd.read_csv(latestdir+'\\Gennembruds_DB\\04_bekræftede_tilfælde_pr_vaccinationsstatus_pr_aldersgrp_pr_uge.csv',delimiter=';',encoding='latin1')
df['Bekræftede tilfælde pr. 100.000'] = pd.to_numeric(df['Bekræftede tilfælde pr. 100.000'].str.replace(',','.'))
df.head(5)


,Uge,Aldersgruppe,Vaccinationsstatus,Bekræftede tilfælde,Befolkningstørrelse,Bekræftede tilfælde pr. 100.000
0,2020-W52,12-15,Forventet fuld effekt,0,0,0.00
1,2020-W52,12-15,Ikke vaccineret,784,283610,276.44
2,2020-W52,12-15,Påbegyndt vaccinationsforløb,0,0,0.00
3,2020-W52,16-19,Forventet fuld effekt,0,0,0.00
4,2020-W52,16-19,Ikke vaccineret,1064,278165,382.51


In [3]:
# Shorten data to use - EVEN SHORTER!
firstWeek = 32
firstWeek = 39
firstIndexToUse = df[df.Uge == f'2021-W{firstWeek}'].index[0]
dfShort = df.loc[firstIndexToUse:]

dfVacc = dfShort[dfShort.Vaccinationsstatus == 'Forventet fuld effekt']
dfUnvacc = dfShort[dfShort.Vaccinationsstatus == 'Ikke vaccineret']

lastWeek = int(df.iloc[-1]['Uge'][-2:])

totNumWeeks = int(lastWeek-firstWeek)
weekRange = np.arange(firstWeek,lastWeek+1)

weekLabels = ['Uge\n'+str(x) for x in weekRange]
weekLabels = ['Uge '+str(x) for x in weekRange]
weekLabels_eng = ['Week '+str(x) for x in weekRange]
weekLabelsSpace_eng = ['     Week '+str(x) for x in weekRange]

allAgeGroups = df.Aldersgruppe.unique()

fullMatUnvaccInci = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatUnvaccCount = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatUnvaccPop = np.zeros((totNumWeeks+1,len(allAgeGroups)))

for ageIndex in range(len(allAgeGroups)):
    curAgeGroup = allAgeGroups[ageIndex]

    agedf = dfUnvacc[(dfUnvacc.Aldersgruppe == curAgeGroup)]


    ageInci = []
    ageCount = []
    agePop = []

    prevPop = 0
    
    # for weekNum in np.arange(firstWeek,lastWeek+1):
    for weekIndex in range(totNumWeeks+1):
        weekNum = weekRange[weekIndex]
        thisdf = agedf[agedf.Uge == str('2021-W'+str(weekNum))]
        
        curInci = thisdf['Bekræftede tilfælde pr. 100.000']
        curCount = thisdf['Bekræftede tilfælde']
        curPop = thisdf['Befolkningstørrelse']
        
        if len(curInci) == 0:
            ageInci.append(0)
            ageCount.append(0)
            # agePop.append(0)
            agePop.append(prevPop)
        else:
            ageInci.append(curInci.values[0])
            ageCount.append(curCount.values[0])
            agePop.append(curPop.values[0])
            prevPop = curPop.values[0]
            
    fullMatUnvaccInci[:,ageIndex] = ageInci
    fullMatUnvaccCount[:,ageIndex] = ageCount
    fullMatUnvaccPop[:,ageIndex] = agePop

fullMatVaccInci = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatVaccCount = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatVaccPop = np.zeros((totNumWeeks+1,len(allAgeGroups)))

for ageIndex in range(len(allAgeGroups)):
    curAgeGroup = allAgeGroups[ageIndex]

    agedf = dfVacc[(dfVacc.Aldersgruppe == curAgeGroup)]


    ageInci = []
    ageCount = []
    agePop = []

    prevPop = 0
    
    # for weekNum in np.arange(firstWeek,lastWeek+1):
    for weekIndex in range(totNumWeeks+1):
        weekNum = weekRange[weekIndex]
        thisdf = agedf[agedf.Uge == str('2021-W'+str(weekNum))]
        
        curInci = thisdf['Bekræftede tilfælde pr. 100.000']
        curCount = thisdf['Bekræftede tilfælde']
        curPop = thisdf['Befolkningstørrelse']
        
        if len(curInci) == 0:
            ageInci.append(0)
            ageCount.append(0)
            # agePop.append(0)
            agePop.append(prevPop)
        else:
            ageInci.append(curInci.values[0])
            ageCount.append(curCount.values[0])
            agePop.append(curPop.values[0])
            prevPop = curPop.values[0]
            
    fullMatVaccInci[:,ageIndex] = ageInci
    fullMatVaccCount[:,ageIndex] = ageCount
    fullMatVaccPop[:,ageIndex] = agePop

In [4]:

levelsToShow = [0,1,2,5,10,20,50,100]
levelsToShow = [0,5,10,20,50,100]
levelsToShow = [0,50,100,200,500,1000]
# hex_colors = sns.light_palette('navy', n_colors=len(levelsToShow) , as_cmap=False).as_hex()
hex_colors = sns.light_palette("#70c030", n_colors=len(levelsToShow) , as_cmap=False).as_hex()
# hex_colors = sns.light_palette('DarkBlue', n_colors=len(levelsToShow) , as_cmap=False).as_hex()
# hex_colors = sns.light_palette(("#ffffff","#0000ff","#000000"), n_colors=len(levelsToShow) , as_cmap=False).as_hex()
# hex_colors = sns.light_palette(, n_colors=len(levelsToShow) , as_cmap=False).as_hex()


new_hex_colors = []
curVal = 0
# prevVal = 0

for k in range(len(levelsToShow)-1):
    # print(k)
    
    curHexColor = hex_colors[k]
    
    nextVal = levelsToShow[k+1] 
    curVal = levelsToShow[k] 
    for n in range(nextVal-curVal):
        new_hex_colors.append(curHexColor)

display(hex_colors)
print(len(new_hex_colors))

['#ecf3e9', '#d3e9c4', '#bade9f', '#a1d47a', '#89ca55', '#70c030']

1000


In [5]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(14,8))


cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

g = sns.heatmap(fullMatVaccInci.transpose(),
                ax=ax1,
                annot=True,
                annot_kws={"fontsize":8},
                fmt='1.0f',
                vmax=levelsToShow[-1],
                vmin=0,
                xticklabels=weekLabelsSpace_eng,
                yticklabels=allAgeGroups,
                cmap=cmap,
                # cbar=False
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax1.collections[0].colorbar
cbar.set_ticks(levelsToShow)
fig.axes[2].set_visible(False)

g = sns.heatmap(fullMatUnvaccInci.transpose(),
                ax=ax2,
                annot=True,
                annot_kws={"fontsize":8},
                fmt='1.0f',
                vmax=levelsToShow[-1],
                vmin=0,
                xticklabels=weekLabelsSpace_eng,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax2.collections[0].colorbar
cbar.set_ticks(levelsToShow)

ax1.set_title('Infections per\n100,000 vaccinated in group.')
ax2.set_title('Infections per\n100,000 unvaccinated in group.')

fig.tight_layout()

SourceText_eng = 'Rasmus K. Pedersen, PandemiX center, Roskilde University - Data-source: covid19.ssi.dk/overvagningsdata \n Filename: "Gennembruds_DB\\04_bekræftede_tilfælde_pr_vaccinationsstatus_pr_aldersgrp_pr_uge.csv"'
ax2.text(18,-1.35,SourceText_eng,ha='right',fontsize=7)

if saveFigures:
    plt.savefig(path_figs+'HeatmapBreakthroughInfections_Both_English')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
plotdf = pd.DataFrame(fullMatVaccInci.transpose())
annodf = pd.DataFrame(fullMatVaccCount.transpose())
annodf2 = pd.DataFrame(fullMatVaccPop.transpose())
annodf3 = pd.DataFrame(fullMatVaccCount.transpose())
annodf3_eng = pd.DataFrame(fullMatVaccCount.transpose())
# display(plotdf)
# print(plotdf.shape)
# display(annodf)
# print(annodf.shape)
# for k in annodf.columns:
#     annodf[k] = 'Antal: '+ annodf[k].astype(int).astype(str)
# for k in annodf2.columns:
#     annodf2[k] = 'Pop: '+ annodf2[k].astype(int).astype(str)
# display(annodf)
# display(annodf2)

for k in annodf3.columns:
    annodf3[k] = annodf[k].astype(int).astype(str) + ' ud af\n' + annodf2[k].astype(int).astype(str)
    annodf3_eng[k] = annodf[k].astype(int).apply(lambda x: "{:,}".format(x)) + '\nout of\n' + annodf2[k].astype(int).apply(lambda x: "{:,}".format(x))
    # annodf3_eng[k] = annodf[k].astype(int).astype(str) + '\nout of\n' + annodf2[k].astype(int).astype(str)
annodf3

plotdf_unvacc = pd.DataFrame(fullMatUnvaccInci.transpose())
annodf_unvacc = pd.DataFrame(fullMatUnvaccCount.transpose())
annodf2_unvacc = pd.DataFrame(fullMatUnvaccPop.transpose())
annodf3_unvacc = pd.DataFrame(fullMatUnvaccCount.transpose())
annodf3_unvacc_eng = pd.DataFrame(fullMatUnvaccCount.transpose())

for k in annodf3_unvacc.columns:
    annodf3_unvacc[k] = annodf_unvacc[k].astype(int).astype(str) + ' ud af\n' + annodf2_unvacc[k].astype(int).astype(str)
    annodf3_unvacc_eng[k] = annodf_unvacc[k].astype(int).apply(lambda x: "{:,}".format(x)) + '\nout of\n' + annodf2_unvacc[k].astype(int).apply(lambda x: "{:,}".format(x))
    # annodf3_unvacc_eng[k] = annodf_unvacc[k].astype(int).astype(str) + '\nout of\n' + annodf2_unvacc[k].astype(int).astype(str)
annodf3_unvacc_eng



,0,1,2,3,4,5,6,7,8,9,10,11
0,"162\nout of\n88,861","223\nout of\n87,968","228\nout of\n87,171","374\nout of\n86,384","509\nout of\n84,942","642\nout of\n82,653","927\nout of\n78,747","1,004\nout of\n73,370","958\nout of\n69,615","1,024\nout of\n66,663","1,324\nout of\n63,790","1,373\nout of\n60,687"
1,"77\nout of\n40,385","77\nout of\n40,156","88\nout of\n39,947","157\nout of\n39,714","249\nout of\n39,431","236\nout of\n39,046","335\nout of\n38,183","391\nout of\n36,529","405\nout of\n35,199","464\nout of\n33,582","535\nout of\n32,118","604\nout of\n31,178"
2,"456\nout of\n335,301","620\nout of\n333,584","698\nout of\n331,706","1,250\nout of\n329,482","1,581\nout of\n326,269","1,765\nout of\n321,885","2,186\nout of\n313,255","2,658\nout of\n299,745","2,533\nout of\n289,824","2,971\nout of\n280,534","3,583\nout of\n272,015","4,463\nout of\n265,404"
3,"242\nout of\n164,680","254\nout of\n163,848","325\nout of\n163,041","488\nout of\n162,248","703\nout of\n161,142","815\nout of\n159,674","1,109\nout of\n156,595","1,343\nout of\n151,643","1,292\nout of\n148,275","1,526\nout of\n145,205","1,783\nout of\n142,306","2,110\nout of\n139,926"
4,"14\nout of\n30,197","16\nout of\n30,077","14\nout of\n29,982","35\nout of\n29,868","51\nout of\n29,702","43\nout of\n29,515","70\nout of\n29,178","99\nout of\n28,722","103\nout of\n28,360","103\nout of\n27,987","120\nout of\n27,614","141\nout of\n27,137"
5,"4\nout of\n9,617","3\nout of\n9,560","10\nout of\n9,523","11\nout of\n9,484","13\nout of\n9,430","14\nout of\n9,379","18\nout of\n9,278","25\nout of\n9,141","31\nout of\n9,036","14\nout of\n8,926","31\nout of\n8,799","29\nout of\n8,617"


In [7]:
display(annodf2_unvacc[k])
annodf2_unvacc[k].astype(int).apply(lambda x: "{:,}".format(x))

0     60687.0
1     31178.0
2    265404.0
3    139926.0
4     27137.0
5      8617.0
Name: 11, dtype: float64

0     60,687
1     31,178
2    265,404
3    139,926
4     27,137
5      8,617
Name: 11, dtype: object

In [9]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(22,14))


cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

g = sns.heatmap(data=plotdf,
                ax=ax1,
                annot=annodf3_eng,
                annot_kws={'va':'top',"fontsize":8},
                fmt='',
                vmax=1000,
                vmin=0,
                xticklabels=weekLabels_eng,
                yticklabels=allAgeGroups,
                cmap=cmap,
                cbar=False,
                )
                

g.invert_yaxis()

g = sns.heatmap(data=plotdf,
                ax=ax1,
                annot=plotdf,
                annot_kws={'va':'bottom',"fontsize":12,'fontweight':'bold'},
                fmt='1.0f',
                vmax=1000,
                vmin=0,
                xticklabels=weekLabels_eng,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax1.collections[1].colorbar
cbar.set_ticks(levelsToShow)
fig.axes[2].set_visible(False)

g = sns.heatmap(data=plotdf_unvacc,
                ax=ax2,
                annot=annodf3_unvacc_eng,
                annot_kws={'va':'top',"fontsize":8},
                fmt='',
                vmax=1000,
                vmin=0,
                xticklabels=weekLabelsSpace_eng,
                yticklabels=allAgeGroups,
                cmap=cmap,
                cbar=False,
                )
                

g.invert_yaxis()

g = sns.heatmap(data=plotdf_unvacc,
                ax=ax2,
                annot=plotdf_unvacc,
                annot_kws={'va':'bottom',"fontsize":12,'fontweight':'bold'},
                fmt='1.0f',
                vmax=1000,
                vmin=0,
                xticklabels=weekLabelsSpace_eng,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax2.collections[1].colorbar
cbar.set_ticks(levelsToShow)

# cbar = ax2.collections[1].colorbar
# cbar.set_ticks(levelsToShow)

    
ax1.set_title('Infections per\n100,000 vaccinated in group.')
ax2.set_title('Infections per\n100,000 unvaccinated in group.')

fig.tight_layout()

SourceText_eng = 'Rasmus K. Pedersen, PandemiX center, Roskilde University - Data-source: covid19.ssi.dk/overvagningsdata \n Filename: "Gennembruds_DB\\04_bekræftede_tilfælde_pr_vaccinationsstatus_pr_aldersgrp_pr_uge.csv"'
# ax2.text(18,-1.35,SourceText_eng,ha='right',fontsize=7)
ax2.text(1,-0.12,SourceText_eng,ha='right',va='bottom',fontsize=7,transform = ax2.transAxes)

if saveFigures:
    plt.savefig(path_figs+'HeatmapBreakthroughInfections_Both_Detailed_English')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …